In [181]:
# Chargement des fichiers Excel depuis les dossiers spécifiques (peu importe leur nom et format)
import pandas as pd
import numpy as np
import re
import os
from datetime import datetime, date
from pathlib import Path
from currency_converter import CurrencyConverter

# Extensions supportées pour les fichiers Excel et CSV
EXCEL_EXTENSIONS = ['.xlsx', '.xlsm', '.xls', '.xlsb', '.xltx', '.xltm', '.xlt', '.csv']

# Fonction pour trouver le fichier Excel/CSV dans un dossier
def find_excel_file(directory):
    """Trouve le premier fichier Excel ou CSV dans le dossier spécifié"""
    if not os.path.exists(directory):
        raise FileNotFoundError(f"Dossier '{directory}' introuvable")
    
    # Chercher tous les fichiers avec les extensions supportées
    excel_files = []
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        if os.path.isfile(file_path):
            file_ext = os.path.splitext(file)[1].lower()
            if file_ext in EXCEL_EXTENSIONS:
                excel_files.append(file)
    
    if len(excel_files) == 0:
        raise FileNotFoundError(f"Aucun fichier Excel/CSV trouvé dans '{directory}'")
    if len(excel_files) > 1:
        raise ValueError(f"Plusieurs fichiers Excel/CSV trouvés dans '{directory}': {excel_files}")
    
    return os.path.join(directory, excel_files[0])

# Fonction pour charger un fichier Excel/CSV selon son extension
def load_excel_file(file_path, header=0):
    """Charge un fichier Excel ou CSV selon son extension
    
    Parameters:
    -----------
    file_path : str
        Chemin vers le fichier à charger
    header : int, default=0
        Numéro de ligne à utiliser comme en-têtes (0 = première ligne)
    """
    file_ext = os.path.splitext(file_path)[1].lower()
    
    if file_ext == '.csv':
        return pd.read_csv(file_path, header=header)
    else:
        return pd.read_excel(file_path, header=header)

# Initialisation du convertisseur de devises
converter = CurrencyConverter()

# Chargement des fichiers depuis les dossiers spécifiques
tab_source_path = find_excel_file('data/Extract LLM de FT')
spire_recap_path = find_excel_file('data/Spire Recap')

ini = load_excel_file(tab_source_path)
df_spire = load_excel_file(spire_recap_path, header=1)  # En-têtes à la ligne 2 (index 1)

# Normaliser les noms de colonnes de df_spire (supprimer espaces, mettre en majuscules)
df_spire.columns = df_spire.columns.str.strip().str.upper()

tableau_final = pd.DataFrame()


ℹ API non disponible - utilisation uniquement du Excel de secours
ℹ Aucun fichier Excel de secours trouvé


In [182]:
# Vérification des colonnes de df_spire pour identifier les noms corrects
print("Colonnes de df_spire:")
print(df_spire.columns.tolist())
print("\nPremières lignes de df_spire:")
df_spire.head()


Colonnes de df_spire:
['N°ISSUANCE', 'CREATION DATE', 'DEALER', 'ISIN']

Premières lignes de df_spire:


,N°ISSUANCE,CREATION DATE,DEALER,ISIN
0,2654,17-Dec-24,NaN,XS2030639145
1,2543,2023-11-18 00:00:00,NaN,XS2135238659
2,2020,25-May-35,NaN,XS2041123880
3,4343,2024-12-18 00:00:00,NaN,XS3205809513


In [183]:
# Définition des dictionnaires de correspondance (dealers, collatéraux)
dealer_name = {
    "BNP Paribas": "BNPP",
}

collat_name = {
    "Republic of Italy": "BTP",
}


In [184]:
# Définition des fonctions utilitaires (conversion devises, formatage dates, parsing formules de taux, etc.)
def convert_to_eur(amount, currency, target_date=None):
    """Convertit un montant vers EUR en utilisant CurrencyConverter"""
    if pd.isna(amount) or pd.isna(currency):
        return np.nan
    currency = str(currency).upper()
    if currency == 'EUR':
        return float(amount)
    if target_date is None:
        target_date = date.today()
    try:
        converted = converter.convert(amount, currency, 'EUR', target_date)
        return converted if converted is not None else np.nan
    except:
        return np.nan

def format_date_dd_mon_yy(date_value):
    if pd.isna(date_value):
        return ""
    try:
        if isinstance(date_value, str):
            date_obj = pd.to_datetime(date_value)
        else:
            date_obj = date_value
        return date_obj.strftime("%d-%b-%y")
    except:
        return ""

def format_percentage(value):
    if pd.isna(value):
        return ""
    try:
        percentage = float(value) * 100
        return f"{percentage:.2f}%"
    except:
        return ""

def parse_rate_formula(formula_str):
    if pd.isna(formula_str):
        return None
    formula = str(formula_str).strip()
    result = {'rate_base': '', 'spread': '', 'floor': '', 'cap': ''}
    
    if 'Floor at' in formula or 'Cap at' in formula:
        rate_match = re.search(r'([A-Z0-9\s\-]+?)\s*\+\s*([\d.]+)%', formula)
        if rate_match:
            result['rate_base'] = rate_match.group(1).strip()
            result['spread'] = rate_match.group(2)
        floor_match = re.search(r'Floor\s+at\s+([\d.]+)%', formula, re.IGNORECASE)
        if floor_match:
            result['floor'] = floor_match.group(1)
        cap_match = re.search(r'Cap\s+at\s+([\d.]+)%', formula, re.IGNORECASE)
        if cap_match:
            result['cap'] = cap_match.group(1)
    elif 'MIN' in formula.upper() and 'MAX' in formula.upper():
        min_match = re.search(r'MIN\s*\[?\s*([\d.]+)%', formula, re.IGNORECASE)
        if min_match:
            result['cap'] = min_match.group(1)
        max_match = re.search(r'MAX\s*\[?\s*([\d.]+)%\s*[;,]?\s*([^)]+)\+?\s*([\d.]+)%?', formula, re.IGNORECASE)
        if max_match:
            result['floor'] = max_match.group(1)
            rate_part = max_match.group(2).strip() if len(max_match.groups()) >= 2 else ''
            spread_part = max_match.group(3) if len(max_match.groups()) >= 3 else ''
            rate_base_match = re.search(r'([A-Z0-9\s\-]+?)(?:\s*\+\s*[\d.]+%?)?$', rate_part)
            if rate_base_match:
                result['rate_base'] = rate_base_match.group(1).strip()
            if spread_part:
                result['spread'] = spread_part
    
    if not result['rate_base']:
        rate_base_match = re.search(r'([A-Z]{2,}[0-9A-Z\s\-]*?)(?:\s*\+\s*[\d.]+%?)?', formula)
        if rate_base_match:
            result['rate_base'] = rate_base_match.group(1).strip()
    
    if 'EURIBOR' in result['rate_base'].upper():
        result['rate_base'] = 'EUR6M'
    
    return result

def format_floating_coupon(floating_note):
    if pd.isna(floating_note):
        return ""
    parsed = parse_rate_formula(floating_note)
    if not parsed:
        return ""
    rate_base = parsed['rate_base'] or 'Rate'
    spread = parsed['spread'] or '0'
    floor = parsed['floor'] or '0'
    cap = parsed['cap'] or ''
    if cap:
        return f"Y1 - End: Min({cap}% ; Max({rate_base} + {spread}% ; {floor}%))"
    else:
        return f"Y1 - End: Max({rate_base} + {spread}% ; {floor}%)"

def format_variable_linked_coupon(fixed_note, variable_note):
    if pd.isna(fixed_note) or pd.isna(variable_note):
        return ""
    parsed = parse_rate_formula(variable_note)
    if not parsed:
        return ""
    rate_base = parsed['rate_base'] or 'Rate'
    spread = parsed['spread'] or '0'
    floor = parsed['floor'] or '0'
    cap = parsed['cap'] or ''
    fixed_str = str(fixed_note)
    if cap:
        return f"Y1 - Y2: {fixed_str}% p.a.\nY2 - End: {fixed_str}% p.a. or Min({cap}% ; Max({rate_base} + {spread}% ; {floor}%)) p.a."
    else:
        return f"Y1 - Y2: {fixed_str}% p.a.\nY2 - End: {fixed_str}% p.a. or Max({rate_base} + {spread}% ; {floor}%) p.a."

def process_pipe_separated(value, formatter=None):
    if pd.isna(value):
        return ""
    value_str = str(value).strip()
    if '|' not in value_str:
        if formatter:
            return formatter(value_str)
        return value_str
    parts = [part.strip() for part in value_str.split('|')]
    if formatter:
        formatted_parts = [formatter(part) for part in parts]
        return " | ".join(formatted_parts)
    else:
        return " | ".join(parts)

def format_number_with_spaces(value):
    """Formate un nombre avec séparateur de milliers (espaces)"""
    if pd.isna(value):
        return ""
    try:
        num = int(float(value))
        return f"{num:,}".replace(",", " ")
    except:
        return ""


In [185]:
# Création colonne Dealer : mapping depuis ini avec remplacement via dictionnaire
tableau_final['Dealer'] = ini['Dealer'].map(lambda x: dealer_name.get(str(x), str(x)) if pd.notna(x) else "")


In [186]:
# Création colonne ISIN (all) : copie directe depuis ini
tableau_final['ISIN (all)'] = ini['ISIN']


In [187]:
# Création colonne N° Issuance : recherche dans df_spire via ISIN pour récupérer l'ID
def get_issuance_number(isin):
    if pd.isna(isin):
        return ""
    # Chercher la colonne ISIN (peut avoir des variations)
    isin_col = None
    for col in df_spire.columns:
        if 'ISIN' in col.upper():
            isin_col = col
            break
    if isin_col is None:
        return ""
    
    matching_row = df_spire[df_spire[isin_col] == isin]
    if not matching_row.empty:
        # Chercher la colonne ID (peut avoir des variations)
        id_col = None
        for col in df_spire.columns:
            if col.upper() == 'ID' or col.upper().strip() == 'ID':
                id_col = col
                break
        if id_col is None:
            return ""
        id_value = matching_row.iloc[0][id_col]
        return str(id_value) if pd.notna(id_value) else ""
    return ""

tableau_final['N° Issuance'] = ini['ISIN'].apply(get_issuance_number)


In [188]:
# Création colonne Creation Date : recherche dans df_spire via ISIN et formatage date DD-Mon-YY
def get_creation_date(isin):
    if pd.isna(isin):
        return ""
    # Chercher la colonne ISIN (peut avoir des variations)
    isin_col = None
    for col in df_spire.columns:
        if 'ISIN' in col.upper():
            isin_col = col
            break
    if isin_col is None:
        return ""
    
    matching_row = df_spire[df_spire[isin_col] == isin]
    if not matching_row.empty:
        # Chercher la colonne Creation Date (peut avoir des variations)
        creation_date_col = None
        for col in df_spire.columns:
            if 'CREATION' in col.upper() and 'DATE' in col.upper():
                creation_date_col = col
                break
        if creation_date_col is None:
            return ""
        creation_date = matching_row.iloc[0][creation_date_col]
        if pd.notna(creation_date):
            return format_date_dd_mon_yy(creation_date)
    return ""

tableau_final['Creation Date'] = ini['ISIN'].apply(get_creation_date)


In [189]:
# Création colonne Maturity : formatage de la date d'échéance au format DD-Mon-YY
tableau_final['Maturity'] = ini['Maturity Date'].apply(format_date_dd_mon_yy)


In [190]:
# Création colonne Currency : copie directe depuis ini
tableau_final['Currency'] = ini['Currency']


In [191]:
# Création colonne Equiv EUR : conversion du nominal en EUR avec formatage séparateur de milliers
tableau_final['Equiv EUR'] = ini.apply(lambda row: format_number_with_spaces(convert_to_eur(row['Nominal'], row['Currency'], row.get('Maturity Date', date.today()))), axis=1)


In [192]:
# Création colonne Issue Price : conversion en pourcentage avec 2 décimales
tableau_final['Issue Price'] = ini['Issue Price'].apply(format_percentage)


In [193]:
# Création colonne Collat Name : mapping via dictionnaire et ajout "I/L" si inflation linked
def process_collat_name(row):
    collat_name_val = row.get('Collat Name', '')
    inflation_linked = row.get('Inflation Linked? (Collat)', '')
    if pd.isna(collat_name_val):
        return ""
    if '|' in str(collat_name_val):
        parts = [p.strip() for p in str(collat_name_val).split('|')]
        mapped_parts = [collat_name.get(part, part) for part in parts]
        return " | ".join(mapped_parts)
    else:
        mapped_name = collat_name.get(str(collat_name_val), str(collat_name_val))
        if pd.notna(inflation_linked) and str(inflation_linked).strip().upper() == 'YES':
            mapped_name += " I/L"
        return mapped_name

tableau_final['Collat Name'] = ini.apply(process_collat_name, axis=1)


In [194]:
# Création colonne Collat ISIN : gestion des valeurs séparées par "|"
tableau_final['Collat ISIN'] = ini['Collat ISIN'].apply(lambda x: process_pipe_separated(x))


In [195]:
# Création colonne Collat CCY : gestion des valeurs séparées par "|"
tableau_final['Collat CCY'] = ini['Collat CCY'].apply(lambda x: process_pipe_separated(x))


In [196]:
# Création colonne Levrage : calcul du levier (collat en EUR / Equiv EUR) en pourcentage
def calculate_leverage_multiple(collat_amount, collat_ccy, nominal, currency, maturity_date=None):
    if pd.isna(collat_amount) or pd.isna(collat_ccy) or pd.isna(nominal) or pd.isna(currency):
        return ""
    if maturity_date is None:
        maturity_date = date.today()
    collat_amount_str = str(collat_amount)
    collat_ccy_str = str(collat_ccy)
    equiv_eur = convert_to_eur(nominal, currency, maturity_date)
    if pd.isna(equiv_eur) or equiv_eur == 0:
        return ""
    if '|' not in collat_amount_str and '|' not in collat_ccy_str:
        collat_eur = convert_to_eur(float(collat_amount), collat_ccy, maturity_date)
        if pd.isna(collat_eur):
            return ""
        leverage = (collat_eur / equiv_eur) * 100
        return f"{leverage:.2f}%"
    amounts = [a.strip() for a in collat_amount_str.split('|')]
    ccies = [c.strip() for c in collat_ccy_str.split('|')]
    leverages = []
    for i, amount in enumerate(amounts):
        if i < len(ccies):
            ccy = ccies[i]
            try:
                collat_eur = convert_to_eur(float(amount), ccy, maturity_date)
                if not pd.isna(collat_eur):
                    leverage = (collat_eur / equiv_eur) * 100
                    leverages.append(f"{leverage:.2f}%")
            except:
                pass
    return " | ".join(leverages) if leverages else ""

tableau_final['Levrage'] = ini.apply(lambda row: calculate_leverage_multiple(
    row.get('Collat Amount', ''),
    row.get('Collat CCY', ''),
    row['Nominal'],
    row['Currency'],
    row.get('Maturity Date', date.today())
), axis=1)


In [197]:
# Création colonne Coupon : formatage selon le type (Fixed, Floating, Variable-linked, Zero Coupon)
def process_coupon(row):
    interest_basis = str(row.get('Interest Basis', '')).strip()
    if interest_basis == 'Fixed':
        fixed_note = row.get('Fixed Note', '')
        if pd.notna(fixed_note):
            return f"Y1 - End: {fixed_note}% p.a."
        return ""
    elif interest_basis == 'Fixed, Variable-linked':
        fixed_note = row.get('Fixed Note', '')
        variable_note = row.get('Variable-linked Note', '')
        return format_variable_linked_coupon(fixed_note, variable_note)
    elif interest_basis == 'Floating':
        floating_note = row.get('Floating Note', '')
        return format_floating_coupon(floating_note)
    elif interest_basis == 'Zero Coupon':
        issue_price = row.get('Issue Price', np.nan)
        maturity_date = row.get('Maturity Date', np.nan)
        issue_date = row.get('Issue Date', np.nan)
        if pd.notna(issue_price) and pd.notna(maturity_date) and pd.notna(issue_date):
            try:
                if isinstance(maturity_date, str):
                    maturity = pd.to_datetime(maturity_date)
                else:
                    maturity = maturity_date
                if isinstance(issue_date, str):
                    issue = pd.to_datetime(issue_date)
                else:
                    issue = issue_date
                years_diff = (maturity - issue).days / 365.25
                issue_price_float = float(issue_price)
                if issue_price_float > 0 and years_diff > 0:
                    irr = -1 + (1 / issue_price_float) ** (1 / years_diff)
                    irr_percent = irr * 100
                    return f"ZC - {irr_percent:.2f}% IRR"
            except Exception as e:
                pass
        return ""
    return ""

tableau_final['Coupon'] = ini.apply(process_coupon, axis=1)


In [198]:
# Création colonne Final Redemption : conversion en pourcentage si nécessaire
def process_final_redemption(value):
    if pd.isna(value):
        return ""
    value_str = str(value).strip()
    if '%' in value_str:
        return value_str
    try:
        num_value = float(value)
        if 0 <= num_value <= 1:
            return f"{num_value * 100:.2f}%"
        elif 1 < num_value <= 100:
            return f"{num_value:.2f}%"
    except:
        pass
    return ""

tableau_final['Final Redemption'] = ini['Final Redemption'].apply(process_final_redemption)


In [199]:
# Création colonne Other comments : Issuer Call et Issuer Switch Option si présents
def process_other_comments(row):
    comments = []
    issuer_call_date = row.get('Issuer Call Redemption Date', '')
    issuer_call_amount = row.get('Issuer Call Redemption Amount', '')
    if pd.notna(issuer_call_date) and str(issuer_call_date).upper() != 'N/A':
        date_str = str(issuer_call_date)
        if pd.notna(issuer_call_amount) and str(issuer_call_amount).upper() != 'N/A':
            try:
                # Convertir le montant en pourcentage entier
                amount_value = float(issuer_call_amount)
                if 0 <= amount_value <= 1:
                    amount_str = f"{int(amount_value * 100)}%"
                elif 1 < amount_value <= 100:
                    amount_str = f"{int(amount_value)}%"
                else:
                    amount_str = str(issuer_call_amount)
            except:
                amount_str = str(issuer_call_amount)
            comments.append(f"Issuer Call {date_str} @{amount_str}")
        else:
            comments.append(f"Issuer Call {date_str}")
    type_of_coupon = row.get('Type of coupon', '')
    if pd.notna(type_of_coupon) and ',' in str(type_of_coupon):
        comments.append("Issuer Switch Option YYYY-MM-DD")
    return " ".join(comments) if comments else ""

tableau_final['Other comments'] = ini.apply(process_other_comments, axis=1)


In [200]:
# Vérification du nombre de colonnes (doit être 15) et affichage du tableau final
if len(tableau_final.columns) != 15:
    import warnings
    warnings.warn(f"ATTENTION: Le tableau final contient {len(tableau_final.columns)} colonnes au lieu de 15 attendues. Colonnes: {list(tableau_final.columns)}")
tableau_final.head()


,Dealer,ISIN (all),N° Issuance,Creation Date,Maturity,Currency,Equiv EUR,Issue Price,Collat Name,Collat ISIN,Collat CCY,Levrage,Coupon,Final Redemption,Other comments
0,BNPP,XS2030639145,,17-Dec-24,15-May-56,EUR,25 000 000,100.00%,BTP I/L,IT0005647273,EUR,98.00%,Y1 - End: Min(6.00% ; Max(EUR6M + 3.14% ; 0.00%)),100%,
1,HSBC Bank plc,XS2135238659,,18-Nov-23,25-Jul-53,EUR,465 000 000,100.00%,Republic of France I/L,FR0014001881,EUR,13.96%,Y1 - End: 4.88% p.a.,100%,Issuer Call 2042-09-15 @100%
2,J.P. Morgan SE,XS2041123880,,25-May-35,15-Sep-42,EUR,123 480 766,75.33%,Basket of Gov,ES0000012E51 | ES0000012932 | ES0000012L60 | E...,EUR | EUR | EUR | EUR,,ZC - 1.69% IRR,,
3,Nomura Financial Products Europe GmbH,XS3205809513,,18-Dec-24,15-Sep-42,EUR,36 481 250,68.53%,The Republic of Italy I/L,IT0005547812,EUR,86.62%,ZC - 2.26% IRR,,
